In [1]:
from __future__ import print_function

import os
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
maxlen = 38  
batch_size = 32

X_test = []
Y_test = []
X_train = []
Y_train = []

POS_PATH = "../../../Documents/Datasets/Twitter/Positive"
NEG_PATH = "../../../Documents/Datasets/Twitter/Negative"

In [3]:
class Preprocessing():
    def __init__(self):
        self.Dict = {}
        self.special = "?!().,،و"
        self.dict_idx = 1
    
    def tokenize(self):
        pass
    
    def normalize(self, text):
        text = ''.join(c for c in text if c not in self.special)
        text = text.replace('أ', 'ا');
        return text
    
    def to_idx(self, data):
        sub_list = []
        for word in data:
            if word not in self.Dict:
                self.Dict[word] = self.dict_idx
                sub_list.append(self.dict_idx)
                self.dict_idx = self.dict_idx + 1
            else:
                sub_list.append(self.Dict[word])
        return sub_list
    
    def features_num(self):
        return self.dict_idx

In [4]:
def read_data(path):
    data = []
    for pos in os.listdir(path):
        file_path = os.path.join(path, pos)
        with open(file_path, encoding="latin1") as file:
            data.append(file.readlines()[0])
    return data

def process_data(data, sentiment=True, split_idx=800):
    for i, sent in enumerate(data):
        sent = text.normalize(sent)
        sent = sent.split()
        if i < split_idx:
            X_train.append(text.to_idx(sent))
            Y_train.append(1 if sentiment else 0)
        else:
            X_test.append(text.to_idx(sent))
            Y_test.append(1 if sentiment else 0)

In [5]:
text = Preprocessing()
pos_data = read_data(POS_PATH)
neg_data = read_data(NEG_PATH)

In [6]:
process_data(pos_data, sentiment=True)
process_data(neg_data, sentiment=False)

In [7]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print(len(X_train))

1600


In [10]:
print('Build model...')
print(text.features_num())
model = Sequential()
model.add(Embedding(text.features_num() + 1, 64))
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Build model...
6861


In [11]:
print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, epochs=20,  validation_data=(X_test, Y_test))

Train...
Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 10s - loss: 0.6880 - acc: 0.5706 - val_loss: 0.6793 - val_acc: 0.6725
Epoch 2/20
1600/1600 [==============================] - 8s - loss: 0.6267 - acc: 0.7206 - val_loss: 0.5813 - val_acc: 0.7575
Epoch 3/20
1600/1600 [==============================] - 8s - loss: 0.4942 - acc: 0.8113 - val_loss: 0.4823 - val_acc: 0.8150
Epoch 4/20
1600/1600 [==============================] - 8s - loss: 0.3744 - acc: 0.8725 - val_loss: 0.4000 - val_acc: 0.8325
Epoch 5/20
1600/1600 [==============================] - 8s - loss: 0.2926 - acc: 0.8888 - val_loss: 0.3620 - val_acc: 0.8475
Epoch 6/20
1600/1600 [==============================] - 8s - loss: 0.2293 - acc: 0.9194 - val_loss: 0.3496 - val_acc: 0.8450
Epoch 7/20
1600/1600 [==============================] - 8s - loss: 0.1671 - acc: 0.9456 - val_loss: 0.3474 - val_acc: 0.8575
Epoch 8/20
1600/1600 [==============================] - 8s - loss: 0

KeyboardInterrupt: 

In [ ]:
score, acc = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)